<a href="https://colab.research.google.com/github/kuds/rl-atari-breakout/blob/main/%5BAtari%20Breakout%5D%20Model-Based%20Reinforcement%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Model-Based Reinforcement Learning to play Atari's Breakout

## References/Repositories
- [Model-Based Reinforcement Learning for Atari - Simplified Repo](https://github.com/dhruvramani/model-based-atari/tree/master)
- [Model-Based Reinforcement Learning for Atari - Paper](https://arxiv.org/abs/1903.00374)
- [Tensor2Tensor - RL Code](https://github.com/tensorflow/tensor2tensor/tree/master/tensor2tensor/rl)

In [39]:
!pip install swig

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.7 MB/s eta 0:00:00


In [40]:
!pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2376096 sha256=889c52bcd641604bf05383e5f35f3f0d4b3fa5cb06ecaef13f73431c287d3b2c
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully built box2d-py


In [55]:
import gymnasium
import platform
import tensorflow
import torch
import numpy
from importlib.metadata import version
from datetime import datetime
import google.colab.drive
import itertools
import multiprocessing
from tensorflow.summary import SummaryWriter

In [56]:
print(f"Python Version: {platform.python_version()}")
print(f"Torch Version: {version('torch')}")
print(f"Is Cuda Available: {torch.cuda.is_available()}")
print(f"Cuda Version: {torch.version.cuda}")
print(f"Gymnasium Version: {version('gymnasium')}")
print(f"Numpy Version: {version('numpy')}")
print(f"Tensorflow Version: {version('tensorflow')}")
print(f"Swig Version: {version('swig')}")

Python Version: 3.10.12
Torch Version: 2.5.0+cu121
Is Cuda Available: False
Cuda Version: 12.1
Gymnasium Version: 1.0.0
Numpy Version: 1.26.4
Tensorflow Version: 2.17.0
Swig Version: 4.2.1.post0


In [57]:
env_str = "CarRacing-v3"
log_dir = "./logs/{}".format(env_str)
env_kwargs_dict={"continuous": False}
gray_scale = True

In [58]:
class Config:
    def __init__(self, **overrides):
        # Default values for all arguments
        self.train_world_model = True
        self.eval_world_model = True
        self.num_rewards = 1
        self.n_envs = 16
        self.is_policy = False
        self.has_rewards = False
        self.hidden_size = 64
        self.n_layers = 6
        self.dropout_p = 0.1
        self.max_ep_len = 500000
        self.l2_clip = 0.0
        self.softmax_clip = 0.03
        self.reward_coeff = 0.1
        self.log_interval = 100
        self.activation_fn = 'relu'
        self.log_dir = './log'
        self.model_dir = './models'
        self.world_model_path = "CarRacingWorldModel"
        self.total_timesteps = int(1e6)
        self.max_eval_iters = int(1e3)
        self.render = True
        self.debug = False

        # Override defaults with any provided values
        self._apply_overrides(overrides)

    def _apply_overrides(self, overrides):
        """Apply any overrides passed in as keyword arguments."""
        for key, value in overrides.items():
            if hasattr(self, key):
                setattr(self, key, value)
            else:
                raise AttributeError(f"Unknown configuration attribute: {key}")

    @classmethod
    def from_argparse(cls):
        """Create a Config instance from command line arguments."""
        parser = argparse.ArgumentParser(
            "Model-Based Reinforcement Learning for Atari",
            formatter_class=argparse.ArgumentDefaultsHelpFormatter
        )

        # Define the same arguments as before
        parser.add_argument('--train_world_model', type=str2bool, default=True)
        parser.add_argument('--eval_world_model', type=str2bool, default=True)
        parser.add_argument('--num_rewards', type=int, default=1)
        parser.add_argument('--n_envs', type=int, default=16)
        parser.add_argument('--is_policy', type=str2bool, default=False)
        parser.add_argument('--has_rewards', type=str2bool, default=False)
        parser.add_argument('--hidden_size', type=int, default=64)
        parser.add_argument('--n_layers', type=int, default=6)
        parser.add_argument('--dropout_p', type=float, default=0.1)
        parser.add_argument('--max_ep_len', type=int, default=500000)
        parser.add_argument('--l2_clip', type=float, default=0.0)
        parser.add_argument('--softmax_clip', type=float, default=0.03)
        parser.add_argument('--reward_coeff', type=float, default=0.1)
        parser.add_argument('--log_interval', type=int, default=100)
        parser.add_argument('--activation_fn', type=str, default='relu', choices=['relu', 'tanh'])
        parser.add_argument('--log_dir', type=str, default='./log')
        parser.add_argument('--model_dir', type=str, default='./models')
        parser.add_argument('--world_model_path', type=str, default="CarRacingWorldModel")
        parser.add_argument('--total_timesteps', type=int, default=int(1e6))
        parser.add_argument('--max_eval_iters', type=int, default=int(1e3))
        parser.add_argument('--render', type=str2bool, default=True, help='Render frames')
        parser.add_argument('--debug', type=str2bool, default=False, help='See debugging info')

        args = parser.parse_args()
        return cls(**vars(args))

    def __repr__(self):
        return f"{self.__class__.__name__}({self.__dict__})"


In [59]:
def printstar(string, num_stars=50):
    print("*" * num_stars)
    print(string)
    print("*" * num_stars)

def make_env():
    def _thunk():
        env = gymnasium.make(env_str)
        return env
    return _thunk

def worker(remote, parent_remote, env_fn_wrapper):
    parent_remote.close()
    env = env_fn_wrapper.x()
    while True:
        cmd, data = remote.recv()
        if cmd == 'step':
            ob, reward, done, info = env.step(data)
            if done:
                ob = env.reset()
            remote.send((ob, reward, done, info))
        elif cmd == 'reset':
            ob = env.reset()
            remote.send(ob)
        elif cmd == 'reset_task':
            ob = env.reset_task()
            remote.send(ob)
        elif cmd == 'close':
            remote.close()
            break
        elif cmd == 'get_spaces':
            remote.send((env.observation_space, env.action_space))
        else:
            raise NotImplementedError

class VecEnv(object):
    """
    An abstract asynchronous, vectorized environment.
    """
    def __init__(self, num_envs, observation_space, action_space):
        self.num_envs = num_envs
        self.observation_space = observation_space
        self.action_space = action_space

    def reset(self):
        """
        Reset all the environments and return an array of
        observations, or a tuple of observation arrays.
        If step_async is still doing work, that work will
        be cancelled and step_wait() should not be called
        until step_async() is invoked again.
        """
        pass

    def step_async(self, actions):
        """
        Tell all the environments to start taking a step
        with the given actions.
        Call step_wait() to get the results of the step.
        You should not call this if a step_async run is
        already pending.
        """
        pass

    def step_wait(self):
        """
        Wait for the step taken with step_async().
        Returns (obs, rews, dones, infos):
         - obs: an array of observations, or a tuple of
                arrays of observations.
         - rews: an array of rewards
         - dones: an array of "episode done" booleans
         - infos: a sequence of info objects
        """
        pass

    def close(self):
        """
        Clean up the environments' resources.
        """
        pass

    def step(self, actions):
        self.step_async(actions)
        return self.step_wait()


class CloudpickleWrapper(object):
    """
    Uses cloudpickle to serialize contents (otherwise multiprocessing tries to use pickle)
    """
    def __init__(self, x):
        self.x = x
    def __getstate__(self):
        import cloudpickle
        return cloudpickle.dumps(self.x)
    def __setstate__(self, ob):
        import pickle
        self.x = pickle.loads(ob)

class SubprocVecEnv(VecEnv):
    def __init__(self, env_fns, spaces=None):
        """
        envs: list of gym environments to run in subprocesses
        """
        self.waiting = False
        self.closed = False
        nenvs = len(env_fns)
        self.nenvs = nenvs
        self.remotes, self.work_remotes = zip(*[mp.Pipe() for _ in range(nenvs)])
        self.ps = [mp.Process(target=worker, args=(work_remote, remote, CloudpickleWrapper(env_fn)))
            for (work_remote, remote, env_fn) in zip(self.work_remotes, self.remotes, env_fns)]
        for p in self.ps:
            p.daemon = True # if the main process crashes, we should not cause things to hang
            p.start()
        for remote in self.work_remotes:
            remote.close()

        self.remotes[0].send(('get_spaces', None))
        observation_space, action_space = self.remotes[0].recv()
        VecEnv.__init__(self, len(env_fns), observation_space, action_space)

    def step_async(self, actions):
        if(type(actions) == int):
            for remote in self.remotes:
                remote.send(('step', actions))
        else:
            for remote, action in zip(self.remotes, actions):
                remote.send(('step', action))
        self.waiting = True

    def step_wait(self):
        results = [remote.recv() for remote in self.remotes]
        self.waiting = False
        obs, rews, dones, infos = zip(*results)
        return np.stack(obs), np.stack(rews), np.stack(dones), infos

    def reset(self):
        for remote in self.remotes:
            remote.send(('reset', None))
        return np.stack([remote.recv() for remote in self.remotes])

    def reset_task(self):
        for remote in self.remotes:
            remote.send(('reset_task', None))
        return np.stack([remote.recv() for remote in self.remotes])

    def close(self):
        if self.closed:
            return
        if self.waiting:
            for remote in self.remotes:
                remote.recv()
        for remote in self.remotes:
            remote.send(('close', None))
        for p in self.ps:
            p.join()
            self.closed = True

    def __len__(self):
        return self.nenvs



def shape_list(x):
  """Return list of dims, statically where possible."""
  x = tf.convert_to_tensor(x)

  # If unknown rank, return dynamic shape
  if x.get_shape().dims is None:
    return tf.shape(x)

  static = x.get_shape().as_list()
  shape = tf.shape(x)

  ret = []
  for i, dim in enumerate(static):
    if dim is None:
      dim = shape[i]
    ret.append(dim)
  return ret

def to_float(x):
  """Cast x to float; created because tf.to_float is deprecated."""
  return tf.cast(x, tf.float32)

def cast_like(x, y):
  """Cast x to y's dtype, if necessary."""
  x = tf.convert_to_tensor(x)
  y = tf.convert_to_tensor(y)

  if x.dtype.base_dtype == y.dtype.base_dtype:
    return x

  cast_x = tf.cast(x, y.dtype)
  if cast_x.device != x.device:
    x_name = "(eager Tensor)"
    try:
      x_name = x.name
    except AttributeError:
      pass
    tf.logging.warning("Cast for %s may induce copy from '%s' to '%s'", x_name,
                       x.device, cast_x.device)
  return cast_x

def layer_norm_vars(filters):
  """Create Variables for layer norm."""
  scale = tf.get_variable(
      "layer_norm_scale", [filters], initializer=tf.ones_initializer())
  bias = tf.get_variable(
      "layer_norm_bias", [filters], initializer=tf.zeros_initializer())
  return scale, bias


def layer_norm_compute(x, epsilon, scale, bias, layer_collection=None):
  """Layer norm raw computation."""

  # Save these before they get converted to tensors by the casting below
  params = (scale, bias)

  epsilon, scale, bias = [cast_like(t, x) for t in [epsilon, scale, bias]]
  mean = tf.reduce_mean(x, axis=[-1], keepdims=True)
  variance = tf.reduce_mean(
      tf.squared_difference(x, mean), axis=[-1], keepdims=True)
  norm_x = (x - mean) * tf.rsqrt(variance + epsilon)

  output = norm_x * scale + bias


  return output

def layer_norm(x,
               filters=None,
               epsilon=1e-6,
               name=None,
               reuse=None,
               layer_collection=None):
  """Layer normalize the tensor x, averaging over the last dimension."""
  if filters is None:
    filters = shape_list(x)[-1]
  with tf.variable_scope(
      name, default_name="layer_norm", values=[x], reuse=reuse):
    scale, bias = layer_norm_vars(filters)
    return layer_norm_compute(x, epsilon, scale, bias,
                              layer_collection=layer_collection)

def standardize_images(x):
  """Image standardization on batches and videos."""
  with tf.name_scope("standardize_images", values=[x]):
    x_shape = shape_list(x)
    x = to_float(tf.reshape(x, [-1] + x_shape[-3:]))
    x_mean = tf.reduce_mean(x, axis=[1, 2], keepdims=True)
    x_variance = tf.reduce_mean(
        tf.squared_difference(x, x_mean), axis=[1, 2], keepdims=True)
    num_pixels = to_float(x_shape[-2] * x_shape[-3])
    x = (x - x_mean) / tf.maximum(tf.sqrt(x_variance), tf.rsqrt(num_pixels))
    return tf.reshape(x, x_shape)

def pad_to_same_length(x, y, final_length_divisible_by=1, axis=1):
  """Pad tensors x and y on axis 1 so that they have the same length."""
  if axis not in [1, 2]:
    raise ValueError("Only axis=1 and axis=2 supported for now.")
  with tf.name_scope("pad_to_same_length", values=[x, y]):
    x_length = shape_list(x)[axis]
    y_length = shape_list(y)[axis]
    if (isinstance(x_length, int) and isinstance(y_length, int) and
        x_length == y_length and final_length_divisible_by == 1):
      return x, y
    max_length = tf.maximum(x_length, y_length)
    if final_length_divisible_by > 1:
      # Find the nearest larger-or-equal integer divisible by given number.
      max_length += final_length_divisible_by - 1
      max_length //= final_length_divisible_by
      max_length *= final_length_divisible_by
    length_diff1 = max_length - x_length
    length_diff2 = max_length - y_length

    def padding_list(length_diff, arg):
      if axis == 1:
        return [[[0, 0], [0, length_diff]],
                tf.zeros([tf.rank(arg) - 2, 2], dtype=tf.int32)]
      return [[[0, 0], [0, 0], [0, length_diff]],
              tf.zeros([tf.rank(arg) - 3, 2], dtype=tf.int32)]

    paddings1 = tf.concat(padding_list(length_diff1, x), axis=0)
    paddings2 = tf.concat(padding_list(length_diff2, y), axis=0)
    res_x = tf.pad(x, paddings1)
    res_y = tf.pad(y, paddings2)
    # Static shapes are the same except for axis=1.
    x_shape = x.shape.as_list()
    x_shape[axis] = None
    res_x.set_shape(x_shape)
    y_shape = y.shape.as_list()
    y_shape[axis] = None
    res_y.set_shape(y_shape)
    return res_x, res_y

def make_even_size(x):
  """Pad x to be even-sized on axis 1 and 2, but only if necessary."""
  x_shape = x.get_shape().as_list()
  assert len(x_shape) > 2, "Only 3+-dimensional tensors supported."
  shape = [dim if dim is not None else -1 for dim in x_shape]
  new_shape = x_shape  # To make sure constant shapes remain constant.
  if x_shape[1] is not None:
    new_shape[1] = 2 * int(math.ceil(x_shape[1] * 0.5))
  if x_shape[2] is not None:
    new_shape[2] = 2 * int(math.ceil(x_shape[2] * 0.5))
  if shape[1] % 2 == 0 and shape[2] % 2 == 0:
    return x
  if shape[1] % 2 == 0:
    x, _ = pad_to_same_length(x, x, final_length_divisible_by=2, axis=2)
    x.set_shape(new_shape)
    return x
  if shape[2] % 2 == 0:
    x, _ = pad_to_same_length(x, x, final_length_divisible_by=2, axis=1)
    x.set_shape(new_shape)
    return x
  x, _ = pad_to_same_length(x, x, final_length_divisible_by=2, axis=1)
  x, _ = pad_to_same_length(x, x, final_length_divisible_by=2, axis=2)
  x.set_shape(new_shape)
  return x


def add_timing_signal_nd(x, min_timescale=1.0, max_timescale=1.0e4):
  """Adds a bunch of sinusoids of different frequencies to a Tensor.

  Each channel of the input Tensor is incremented by a sinusoid of a different
  frequency and phase in one of the positional dimensions.

  This allows attention to learn to use absolute and relative positions.
  Timing signals should be added to some precursors of both the query and the
  memory inputs to attention.

  The use of relative position is possible because sin(a+b) and cos(a+b) can be
  expressed in terms of b, sin(a) and cos(a).

  x is a Tensor with n "positional" dimensions, e.g. one dimension for a
  sequence or two dimensions for an image

  We use a geometric sequence of timescales starting with
  min_timescale and ending with max_timescale.  The number of different
  timescales is equal to channels // (n * 2). For each timescale, we
  generate the two sinusoidal signals sin(timestep/timescale) and
  cos(timestep/timescale).  All of these sinusoids are concatenated in
  the channels dimension.

  Args:
    x: a Tensor with shape [batch, d1 ... dn, channels]
    min_timescale: a float
    max_timescale: a float

  Returns:
    a Tensor the same shape as x.
  """
  num_dims = len(x.get_shape().as_list()) - 2
  channels = shape_list(x)[-1]
  num_timescales = channels // (num_dims * 2)
  log_timescale_increment = (
      math.log(float(max_timescale) / float(min_timescale)) /
      (tf.to_float(num_timescales) - 1))
  inv_timescales = min_timescale * tf.exp(
      tf.to_float(tf.range(num_timescales)) * -log_timescale_increment)
  for dim in range(num_dims):
    length = shape_list(x)[dim + 1]
    position = tf.to_float(tf.range(length))
    scaled_time = tf.expand_dims(position, 1) * tf.expand_dims(
        inv_timescales, 0)
    signal = tf.concat([tf.sin(scaled_time), tf.cos(scaled_time)], axis=1)
    prepad = dim * 2 * num_timescales
    postpad = channels - (dim + 1) * 2 * num_timescales
    signal = tf.pad(signal, [[0, 0], [prepad, postpad]])
    for _ in range(1 + dim):
      signal = tf.expand_dims(signal, 0)
    for _ in range(num_dims - 1 - dim):
      signal = tf.expand_dims(signal, -2)
    x += signal
  return x


In [60]:
g_env_model = None

def cached_world_model(ob_shape, action_dim, config, path):
    global g_env_model
    if g_env_model is None:
        old_val = config.n_envs
        config.n_envs = 1
        g_env_model = EnvModel(ob_shape, action_dim, config)
        g_env_model.load_state_dict(torch.load(path))
        g_env_model.eval()
        printstar('World Model Restored')
        config.n_envs = old_val

    return g_env_model

class EnvModel(nn.Module):
    def __init__(self, obs_shape, action_dim, config):
        super(EnvModel, self).__init__()
        self.obs_shape = obs_shape
        self.action_dim = action_dim
        self.config = config

        self.hidden_size = config.hidden_size
        self.layers = config.n_layers
        self.dropout_p = config.dropout_p

        self.activation_fn = nn.ReLU() if config.activation_fn == 'relu' else nn.Tanh()

        self.l2_clip = config.l2_clip
        self.softmax_clip = config.softmax_clip
        self.reward_coeff = config.reward_coeff
        self.n_envs = config.n_envs
        self.max_ep_len = config.max_ep_len
        self.log_interval = config.log_interval

        self.has_rewards = config.has_rewards
        self.num_rewards = config.num_rewards

        self.width, self.height, self.depth = self.obs_shape

        # Define model architecture
        self.encoder = nn.Sequential(
            nn.Conv2d(self.depth, self.hidden_size, kernel_size=3, stride=1, padding=1),
            self.activation_fn,
            nn.Dropout(self.dropout_p)
        )
        self.middle_layers = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(self.hidden_size, self.hidden_size, kernel_size=3, stride=1, padding=1),
                self.activation_fn,
                nn.Dropout(self.dropout_p)
            ) for _ in range(self.layers)
        ])
        self.decoder = nn.Conv2d(self.hidden_size, self.depth, kernel_size=3, stride=1, padding=1)

        self.reward_predictor = None
        if self.has_rewards:
            self.reward_predictor = nn.Sequential(
                nn.Linear(self.hidden_size, 128),
                nn.ReLU(),
                nn.Linear(128, self.num_rewards)
            )

    def forward(self, obs, action):
        # Standardize images
        obs = standardize_images(obs)
        x = self.encoder(obs)
        for layer in self.middle_layers:
            x = x + layer(x)
        state_pred = self.decoder(x)

        reward_pred = None
        if self.has_rewards:
            x_flat = torch.mean(x, dim=[2, 3])
            reward_pred = self.reward_predictor(x_flat)

        return state_pred, reward_pred

    def imagine(self, obs, action):
        obs = obs.unsqueeze(0).permute(0, 3, 1, 2)  # Add batch dimension, switch to (B, C, H, W)
        action = torch.tensor(action, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            next_pred_ob, _ = self.forward(obs, action)
        return next_pred_ob.squeeze(0).permute(1, 2, 0).numpy()  # Back to (H, W, C)

    def train_model(self, envs, config, world_model_path):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        loss_fn = nn.MSELoss()

        writer = SummaryWriter(log_dir='./env_logs/train')

        for idx, states, actions, rewards, next_states, dones in tqdm(
            self.generate_data(envs), total=self.max_ep_len):

            optimizer.zero_grad()

            # Convert inputs to tensors
            states = torch.tensor(states, dtype=torch.float32).permute(0, 3, 1, 2)
            actions = torch.tensor(actions, dtype=torch.float32)
            next_states = torch.tensor(next_states, dtype=torch.float32).permute(0, 3, 1, 2)

            state_pred, reward_pred = self.forward(states, actions)

            state_loss = torch.clamp(loss_fn(state_pred, next_states), max=self.l2_clip)

            loss = state_loss
            if self.has_rewards:
                rewards = torch.tensor(rewards, dtype=torch.float32)
                reward_loss = torch.clamp(loss_fn(reward_pred, rewards), max=self.softmax_clip)
                loss += self.reward_coeff * reward_loss

            loss.backward()
            optimizer.step()

            if idx % config.log_interval == 0:
                print(f'{idx} => Loss: {loss.item():.4f}')
                torch.save(self.state_dict(), os.path.join(world_model_path, 'env_model.pt'))
                print('Environment model saved')

                writer.add_scalar('Loss/Total', loss.item(), idx)
        writer.close()

    def generate_data(self, envs):
        states = envs.reset()
        for frame_idx in range(self.max_ep_len):
            states = states.reshape(self.n_envs, self.width, self.height, self.depth)
            actions = [envs.action_space.sample() for _ in range(self.n_envs)]
            next_states, rewards, dones, _ = envs.step(actions)
            next_states = next_states.reshape(self.n_envs, self.width, self.height, self.depth)

            yield frame_idx, states, actions, rewards, next_states, dones
            states = next_states
            if self.n_envs == 1 and dones:
                states = envs.reset()


In [61]:
g_env_model = None

def main(config):
    global make_env
    env = make_env()()
    env.reset()

    action_dim = env.action_space.shape[0]  # Assuming continuous action space
    ob_shape = env.observation_space.shape
    world_model_path = os.path.expanduser(os.path.join(config.model_dir, config.world_model_path))

    env_model = EnvModel(ob_shape, action_dim, config)
    optimizer = optim.Adam(env_model.parameters(), lr=0.001)

    if config.train_world_model:
        if not os.path.exists(world_model_path):
            os.makedirs(world_model_path)
        printstar("Training World Model")
        env_model.train_model(env, config, optimizer)
        torch.save(env_model.state_dict(), os.path.join(world_model_path, 'env_model.pt'))

    if config.eval_world_model:
        env_model.load_state_dict(torch.load(os.path.join(world_model_path, 'env_model.pt')))
        evaluate_world_model(env, env_model, config)

def evaluate_world_model(env, env_model, config, policy=None):
    printstar("Testing World Model")
    env_model.eval()
    obs = env.reset()

    with torch.no_grad():
        for t in range(config.max_eval_iters):
            action = policy(obs) if policy else env.action_space.sample()
            action_tensor = torch.tensor(action, dtype=torch.float32).unsqueeze(0)
            obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)

            next_pred_ob = env_model(obs_tensor, action_tensor).squeeze(0).numpy()
            plt.imshow(next_pred_ob.reshape(env.observation_space.shape[:2]))
            plt.savefig('./figs/world_model_eval.png')

            env.render()
            obs, _, done, _ = env.step(action)
            if input("Press 0 to exit: ") == "0":
                break
            if done:
                obs = env.reset()

# Configure and run in the notebook
if __name__ == '__main__':
    config = Config(train_world_model=True, eval_world_model=True, debug=False)  # Update as needed
    main(config)


**************************************************
Training World Model
**************************************************


TypeError: SummaryWriter() takes no arguments

In [ ]:
# main.py

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from world_model import *
from config import argparser
from utils import make_env as env_fn, printstar

g_env_model = None

def main(config):
    global env_fn
    env = env_fn()()
    env.reset()

    action_dim = 5
    ob_shape = env.observation_space.shape
    world_model_path = os.path.expanduser(os.path.join(config.model_dir, config.world_model_type + "_" + config.world_model_path))

    if(config.train_world_model):
        env_model = EnvModel(ob_shape, action_dim, config)
        if(not os.path.exists(world_model_path)):
            os.mkdir(world_model_path)
        printstar("Training World Model")
        env_model.train(world_model_path)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        if(config.eval_world_model):
            env_model = cached_world_model(sess, ob_shape, action_dim, config, world_model_path + '/env_model.ckpt')
            evaluate_world_model(env, sess, env_model, config)

def evaluate_world_model(env, sess, env_model, config, policy=None):
    printstar("Testing World Model")
    obs = env.reset()
    for t in range(config.max_eval_iters):
        if(policy is None):
                action = env.action_space.sample()
        else:
            action = policy(obs)

        next_pred_ob = env_model.imagine(sess, obs, action)
        imgplot = plt.imshow(next_pred_ob)
        plt.savefig('./figs/world_model_eval.png')

        env.render()
        obs, reward, dones, info = env.step(action)
        inp = input("Press 0 to exit : ")
        if(inp == "0"):
            break

if __name__ == '__main__':
    config = argparser()
    mp.set_start_method('spawn', force=True)
    main(config)

In [ ]:
def collect_data(env, num_episodes=1000):
    data = []
    for _ in range(num_episodes):
        state = env.reset()
        terminated = False
        truncated = False
        while not (terminated or truncated):
            action = env.action_space.sample()
            next_state, reward, terminated, truncated, info = env.step(action)
            data.append((state, action, reward, next_state))
            if(terminated or truncated):
              print(next_state)
            state = next_state
    return data

# Collect data

data = collect_data(env)
print(len(data))

In [ ]:
print(data[0][0])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network for the dynamics model
class DynamicsModel(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DynamicsModel, self).__init__()
        self.fc1 = nn.Linear(state_dim + action_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, state_dim)  # Predict next state
        self.reward = nn.Linear(128, 1)       # Predict reward

    def forward(self, state, action):
        x = torch.cat([state, action], dim=-1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        next_state = self.fc3(x)
        reward = self.reward(x)
        return next_state, reward


In [ ]:
# Initialize model and optimizer
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
model = DynamicsModel(state_dim, action_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Convert collected data to tensors for training
states = torch.tensor([d[0] for d in data], dtype=torch.float32)
actions = torch.tensor([d[1] for d in data], dtype=torch.float32).unsqueeze(1)
next_states = torch.tensor([d[3] for d in data], dtype=torch.float32)
rewards = torch.tensor([d[2] for d in data], dtype=torch.float32).unsqueeze(1)

# Train the dynamics model
for epoch in range(100):
    optimizer.zero_grad()
    predicted_next_states, predicted_rewards = model(states, actions)
    loss = criterion(predicted_next_states, next_states) + criterion(predicted_rewards, rewards)
    loss.backward()
    optimizer.step()

print("Model training complete!")


In [ ]:
[d[0] for d in data]

In [ ]:
def mpc_action_selection(model, current_state, num_simulations=100, horizon=10):
    best_action = None
    best_reward = -np.inf

    for _ in range(num_simulations):
        simulated_state = current_state
        total_reward = 0
        for _ in range(horizon):
            action = np.random.choice([0, 1])  # Random action sampling for now
            action_tensor = torch.tensor([action], dtype=torch.float32).unsqueeze(0)
            state_tensor = torch.tensor(simulated_state, dtype=torch.float32).unsqueeze(0)
            next_state, reward = model(state_tensor, action_tensor)
            total_reward += reward.item()
            simulated_state = next_state.detach().numpy()[0]

        if total_reward > best_reward:
            best_reward = total_reward
            best_action = action

    return best_action


In [ ]:
def evaluate_model_based_agent(env, model, num_episodes=10):
    for episode in range(num_episodes):
        state = env.reset()
        done = False
        total_reward = 0
        while not done:
            action = mpc_action_selection(model, state)
            state, reward, done, _ = env.step(action)
            total_reward += reward
        print(f"Episode {episode + 1}: Total Reward: {total_reward}")

# Evaluate the agent
evaluate_model_based_agent(env, model)


In [ ]:
# world_model.py

import os
import gym
import numpy as np
from tqdm import tqdm
import tensorflow as tf

from utils import *

g_env_model = None
def cached_world_model(sess, ob_shape, action_dim, config, path):
    global g_env_model
    if g_env_model is None:
        old_val = config.n_envs
        config.n_envs = 1
        g_env_model = EnvModel(ob_shape, action_dim, config)
        save_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='env_model')
        loader = tf.train.Saver(var_list=save_vars)
        loader.restore(sess, path)
        printstar('World Model Restored')
        config.n_envs = old_val

    return g_env_model

def inject_additional_input(layer, inputs, name, mode="multi_additive"):
  """Injects the additional input into the layer.

  Args:
    layer: layer that the input should be injected to.
    inputs: inputs to be injected.
    name: TF scope name.
    mode: how the infor should be added to the layer:
      "concat" concats as additional channels.
      "multiplicative" broadcasts inputs and multiply them to the channels.
      "multi_additive" broadcasts inputs and multiply and add to the channels.

  Returns:
    updated layer.

  Raises:
    ValueError: in case of unknown mode.
  """
  layer_shape = shape_list(layer)
  input_shape = shape_list(inputs)
  zeros_mask = tf.zeros(layer_shape, dtype=tf.float32)
  if mode == "concat":
    emb = common_video.encode_to_shape(inputs, layer_shape, name)
    layer = tf.concat(values=[layer, emb], axis=-1)
  elif mode == "multiplicative":
    filters = layer_shape[-1]
    input_reshaped = tf.reshape(inputs, [-1, 1, 1, input_shape[-1]])
    input_mask = tf.layers.dense(input_reshaped, filters, name=name)
    input_broad = input_mask + zeros_mask
    layer *= input_broad
  elif mode == "multi_additive":
    filters = layer_shape[-1]
    input_reshaped = tf.reshape(inputs, [-1, 1, 1, input_shape[-1]])
    input_mul = tf.layers.dense(input_reshaped, filters, name=name + "_mul")
    layer *= tf.nn.sigmoid(input_mul)
    input_add = tf.layers.dense(input_reshaped, filters, name=name + "_add")
    layer += input_add
  else:
    raise ValueError("Unknown injection mode: %s" % mode)

  return layer

class EnvModel(object):
    def __init__(self, obs_shape, action_dim, config):

        self.obs_shape = obs_shape
        self.action_dim = action_dim
        self.config = config

        self.hidden_size = config.hidden_size
        self.layers = config.n_layers
        self.dropout_p = config.dropout_p

        if(config.activation_fn == 'relu'):
            self.activation_fn = tf.nn.relu
        elif (config.activation_fn == 'tanh'):
            self.activation_fn = tf.nn.tanh

        self.l2_clip = config.l2_clip
        self.softmax_clip = config.softmax_clip
        self.reward_coeff = config.reward_coeff
        self.n_envs = config.n_envs
        self.max_ep_len = config.max_ep_len
        self.log_interval = config.log_interval

        self.is_policy = config.is_policy
        self.has_rewards = config.has_rewards
        self.num_rewards = config.num_rewards

        self.width, self.height, self.depth = self.obs_shape

        self.states_ph = tf.placeholder(tf.float32, [None, self.width, self.height, self.depth])
        self.actions_ph = tf.placeholder(tf.uint8, [None, 1])
        self.actions_oph = tf.one_hot(self.actions_ph, depth=action_dim)
        self.target_states = tf.placeholder(tf.float32, [None, self.width, self.height, self.depth])
        if(self.has_rewards):
            self.target_rewards = tf.placeholder(tf.uint8, [None, self.num_rewards])

        # NOTE - Implement policy and value parts later
        with tf.variable_scope("env_model"):
            self.state_pred, self.reward_pred, _, _ = self.network()

        # NOTE - Change this maybe to video_l2_loss
        self.state_loss = tf.math.maximum(tf.reduce_sum(tf.pow(self.state_pred - self.target_states, 2)), self.l2_clip)
        self.loss = self.state_loss

        if(self.has_rewards):
            self.reward_loss = tf.math.maximum(tf.reduce_mean(tf.losses.softmax_cross_entropy(self.tw_one_hot, self.reward_pred)), self.softmax_clip)
            self.loss = self.loss + (self.reward_coeff * self.reward_loss)

        self.opt = tf.train.AdamOptimizer().minimize(self.loss)

        tf.summary.scalar('loss', self.loss)
        if(self.has_rewards):
            tf.summary.scalar('image_loss', self.state_loss)
            tf.summary.scalar('reward_loss', self.reward_loss)

    def generate_data(self, envs):
        states = envs.reset()
        for frame_idx in range(self.max_ep_len):
            states = states.reshape(self.n_envs, self.width, self.height, self.depth)
            if(self.n_envs == 1):
                actions = envs.action_space.sample()
            else:
                actions = [envs.action_space.sample() for _ in range(self.n_envs)]
            next_states, rewards, dones, _ = envs.step(actions)
            next_states = next_states.reshape(self.n_envs, self.width, self.height, self.depth)

            yield frame_idx, states, actions, rewards, next_states, dones
            states = next_states
            if(self.n_envs == 1 and dones == True):
                states = envs.reset()

    def network(self):
        def middle_network(layer):
            x = layer
            kernel1 = (3, 3)
            filters = shape_list(x)[-1]
            for i in range(2):
              with tf.variable_scope("layer%d" % i):
                y = tf.nn.dropout(x, 1.0 - 0.5)
                y = tf.layers.conv2d(y, filters, kernel1, activation=self.activation_fn,
                                     strides=(1, 1), padding="SAME")
                if i == 0:
                  x = y
                else:
                  x = layer_norm(x + y)
            return x

        batch_size = tf.shape(self.states_ph)[0]

        filters = self.hidden_size
        kernel2 = (4, 4)
        action = self.actions_oph

        # Normalize states
        if(self.n_envs > 1):
            states = [standardize_images(self.states_ph[i, :, :, :]) for i in range(self.n_envs)]
            stacked_states = tf.stack(states)
        else:
            stacked_states = standardize_images(self.states_ph)
        inputs_shape = shape_list(stacked_states)

        # Using non-zero bias initializer below for edge cases of uniform inputs.
        x = tf.layers.dense(
            stacked_states, filters, name="inputs_embed",
            bias_initializer=tf.random_normal_initializer(stddev=0.01))
        x = add_timing_signal_nd(x)

        # Down-stride.
        layer_inputs = [x]
        for i in range(self.layers):
          with tf.variable_scope("downstride%d" % i):
            layer_inputs.append(x)
            x = tf.nn.dropout(x, 1.0 - self.dropout_p)
            x = make_even_size(x)
            if i < 2:
              filters *= 2
            x = add_timing_signal_nd(x)
            x = tf.layers.conv2d(x, filters, kernel2, activation=self.activation_fn,
                                 strides=(2, 2), padding="SAME")
            x = layer_norm(x)

        if self.is_policy:
          with tf.variable_scope("policy"):
            x_flat = tf.layers.flatten(x)
            policy_pred = tf.layers.dense(x_flat, self.action_dim)
            value_pred = tf.layers.dense(x_flat, 1)
            value_pred = tf.squeeze(value_pred, axis=-1)
        else:
          policy_pred, value_pred = None, None

        x = inject_additional_input(x, action, "action_enc", "multi_additive")

        # Inject latent if present. Only for stochastic models.
        target_states = standardize_images(self.target_states)

        x_mid = tf.reduce_mean(x, axis=[1, 2], keepdims=True)
        x = middle_network(x)

        # Up-convolve.
        layer_inputs = list(reversed(layer_inputs))
        for i in range(self.layers):
          with tf.variable_scope("upstride%d" % i):
            x = tf.nn.dropout(x, 1.0 - 0.1)
            if i >= self.layers - 2:
              filters //= 2
            x = tf.layers.conv2d_transpose(
                x, filters, kernel2, activation=self.activation_fn,
                strides=(2, 2), padding="SAME")
            y = layer_inputs[i]
            shape = shape_list(y)
            x = x[:, :shape[1], :shape[2], :]
            x = layer_norm(x + y)
            x = add_timing_signal_nd(x)

        # Cut down to original size.
        x = x[:, :inputs_shape[1], :inputs_shape[2], :]
        x_fin = tf.reduce_mean(x, axis=[1, 2], keepdims=True)

        x = tf.layers.dense(x, self.depth, name="logits")

        reward_pred = None
        if self.has_rewards:
          # Reward prediction based on middle and final logits.
          reward_pred = tf.concat([x_mid, x_fin], axis=-1)
          reward_pred = tf.nn.relu(tf.layers.dense(
              reward_pred, 128, name="reward_pred"))
          reward_pred = tf.squeeze(reward_pred, axis=1)  # Remove extra dims
          reward_pred = tf.squeeze(reward_pred, axis=1)  # Remove extra dims

        return x, reward_pred, policy_pred, value_pred

    def imagine(self, sess, obs, action):
        action = np.array(action)
        action = np.reshape(action, (1, 1))
        obs = obs.reshape(1, self.width, self.height, self.depth)
        next_pred_ob = sess.run(self.state_pred, feed_dict={self.states_ph : obs, self.actions_ph : action})
        next_pred_ob = next_pred_ob.reshape(self.width, self.height, self.depth)
        next_pred_ob = np.rint(next_pred_ob)
        return next_pred_ob

    def train(self, world_model_path):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            losses = []
            all_rewards = []
            save_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='env_model')
            saver = tf.train.Saver(var_list=save_vars)

            train_writer = tf.summary.FileWriter('./env_logs/train/', graph=sess.graph)
            summary_op = tf.summary.merge_all()

            if(self.n_envs == 1):
                envs = make_env()()
            else:
                envs = [make_env() for i in range(self.n_envs)]
                envs = SubprocVecEnv(envs)

            for idx, states, actions, rewards, next_states, dones in tqdm(
                self.generate_data(envs), total=self.max_ep_len):
                actions = np.array(actions)
                actions = np.reshape(actions, (-1, 1))

                if(self.has_rewards):
                    target_reward = reward_to_target(rewards)
                    loss, reward_loss, state_loss, summary, _ = sess.run([self.loss, self.reward_loss, self.state_loss,
                        summary_op, self.opt], feed_dict={
                        self.states_ph: states,
                        self.actions_ph: actions,
                        self.target_states: next_states,
                        self.target_rewards: target_reward
                    })
                else :
                    loss, summary, _ = sess.run([self.loss, summary_op, self.opt], feed_dict={
                        self.states_ph: states,
                        self.actions_ph: actions,
                        self.target_states: next_states,
                    })

                if idx % self.log_interval == 0:
                    if(self.has_rewards):
                        print('%i => Loss : %.4f, Reward Loss : %.4f, Image Loss : %.4f' % (idx, loss, reward_loss, state_loss))
                    else :
                        print('%i => Loss : %.4f' % (idx, loss))
                    saver.save(sess, '{}/env_model.ckpt'.format(world_model_path))
                    print('Environment model saved')

                train_writer.add_summary(summary, idx)
            envs.close()

In [ ]:
# main.py

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from world_model import *
from config import argparser
from utils import make_env as env_fn, printstar

g_env_model = None

def main(config):
    global env_fn
    env = env_fn()()
    env.reset()

    action_dim = 5
    ob_shape = env.observation_space.shape
    world_model_path = os.path.expanduser(os.path.join(config.model_dir, config.world_model_type + "_" + config.world_model_path))

    if(config.train_world_model):
        env_model = EnvModel(ob_shape, action_dim, config)
        if(not os.path.exists(world_model_path)):
            os.mkdir(world_model_path)
        printstar("Training World Model")
        env_model.train(world_model_path)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        if(config.eval_world_model):
            env_model = cached_world_model(sess, ob_shape, action_dim, config, world_model_path + '/env_model.ckpt')
            evaluate_world_model(env, sess, env_model, config)

def evaluate_world_model(env, sess, env_model, config, policy=None):
    printstar("Testing World Model")
    obs = env.reset()
    for t in range(config.max_eval_iters):
        if(policy is None):
                action = env.action_space.sample()
        else:
            action = policy(obs)

        next_pred_ob = env_model.imagine(sess, obs, action)
        imgplot = plt.imshow(next_pred_ob)
        plt.savefig('./figs/world_model_eval.png')

        env.render()
        obs, reward, dones, info = env.step(action)
        inp = input("Press 0 to exit : ")
        if(inp == "0"):
            break

if __name__ == '__main__':
    config = argparser()
    mp.set_start_method('spawn', force=True)
    main(config)

In [ ]:
# config.py

import os
import argparse

def str2bool(v):
    return v.lower() == 'true'

def str2list(v):
    if not v:
        return v
    else:
        return [v_ for v_ in v.split(',')]

def argparser():
    parser = argparse.ArgumentParser("Model-Based Reinforcement Learning for Atari",
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)

    parser.add_argument('--train_world_model', type=str2bool, default=True)
    parser.add_argument('--eval_world_model', type=str2bool, default=True)

    parser.add_argument('--num_rewards', type=int, default=1)
    parser.add_argument('--n_envs', type=int, default=16)
    parser.add_argument('--is_policy', type=str2bool, default=False)
    parser.add_argument('--has_rewards', type=str2bool, default=False)
    parser.add_argument('--hidden_size', type=int, default=64)
    parser.add_argument('--n_layers', type=int, default=6)
    parser.add_argument('--dropout_p', type=float, default=0.1)
    parser.add_argument('--max_ep_len', type=int, default=500000)
    parser.add_argument('--l2_clip', type=float, default=0.0)
    parser.add_argument('--softmax_clip', type=float, default=0.03)
    parser.add_argument('--reward_coeff', type=float, default=0.1)
    parser.add_argument('--log_interval', type=int, default=100)
    parser.add_argument('--activation_fn', type=str, default='relu', choices=['relu', 'tanh'])

    parser.add_argument('--log_dir', type=str, default='./log')
    parser.add_argument('--model_dir', type=str, default='./models')
    parser.add_argument('--world_model_path', type=str, default="CarRacingWorldModel")

    parser.add_argument('--total_timesteps', type=int, default=int(1e6))
    parser.add_argument('--max_eval_iters', type=int, default=int(1e3))

    parser.add_argument('--render', type=str2bool, default=True, help='Render frames')
    parser.add_argument('--debug', type=str2bool, default=False, help='See debugging info')

    args = parser.parse_args()
    return args

In [ ]:
# utils.py
# Code is from OpenAI Baseline and Tensor2Tensor

import itertools
import numpy as np
from gym.envs.box2d import CarRacing
import multiprocessing as mp

def printstar(string, num_stars=50):
    print("*" * num_stars)
    print(string)
    print("*" * num_stars)

def make_env():
    def _thunk():
        env = CarRacing(grayscale=0, show_info_panel=0, discretize_actions="hard", frames_per_state=1, num_lanes=1, num_tracks=1)
        return env
    return _thunk

def worker(remote, parent_remote, env_fn_wrapper):
    parent_remote.close()
    env = env_fn_wrapper.x()
    while True:
        cmd, data = remote.recv()
        if cmd == 'step':
            ob, reward, done, info = env.step(data)
            if done:
                ob = env.reset()
            remote.send((ob, reward, done, info))
        elif cmd == 'reset':
            ob = env.reset()
            remote.send(ob)
        elif cmd == 'reset_task':
            ob = env.reset_task()
            remote.send(ob)
        elif cmd == 'close':
            remote.close()
            break
        elif cmd == 'get_spaces':
            remote.send((env.observation_space, env.action_space))
        else:
            raise NotImplementedError

class VecEnv(object):
    """
    An abstract asynchronous, vectorized environment.
    """
    def __init__(self, num_envs, observation_space, action_space):
        self.num_envs = num_envs
        self.observation_space = observation_space
        self.action_space = action_space

    def reset(self):
        """
        Reset all the environments and return an array of
        observations, or a tuple of observation arrays.
        If step_async is still doing work, that work will
        be cancelled and step_wait() should not be called
        until step_async() is invoked again.
        """
        pass

    def step_async(self, actions):
        """
        Tell all the environments to start taking a step
        with the given actions.
        Call step_wait() to get the results of the step.
        You should not call this if a step_async run is
        already pending.
        """
        pass

    def step_wait(self):
        """
        Wait for the step taken with step_async().
        Returns (obs, rews, dones, infos):
         - obs: an array of observations, or a tuple of
                arrays of observations.
         - rews: an array of rewards
         - dones: an array of "episode done" booleans
         - infos: a sequence of info objects
        """
        pass

    def close(self):
        """
        Clean up the environments' resources.
        """
        pass

    def step(self, actions):
        self.step_async(actions)
        return self.step_wait()


class CloudpickleWrapper(object):
    """
    Uses cloudpickle to serialize contents (otherwise multiprocessing tries to use pickle)
    """
    def __init__(self, x):
        self.x = x
    def __getstate__(self):
        import cloudpickle
        return cloudpickle.dumps(self.x)
    def __setstate__(self, ob):
        import pickle
        self.x = pickle.loads(ob)

class SubprocVecEnv(VecEnv):
    def __init__(self, env_fns, spaces=None):
        """
        envs: list of gym environments to run in subprocesses
        """
        self.waiting = False
        self.closed = False
        nenvs = len(env_fns)
        self.nenvs = nenvs
        self.remotes, self.work_remotes = zip(*[mp.Pipe() for _ in range(nenvs)])
        self.ps = [mp.Process(target=worker, args=(work_remote, remote, CloudpickleWrapper(env_fn)))
            for (work_remote, remote, env_fn) in zip(self.work_remotes, self.remotes, env_fns)]
        for p in self.ps:
            p.daemon = True # if the main process crashes, we should not cause things to hang
            p.start()
        for remote in self.work_remotes:
            remote.close()

        self.remotes[0].send(('get_spaces', None))
        observation_space, action_space = self.remotes[0].recv()
        VecEnv.__init__(self, len(env_fns), observation_space, action_space)

    def step_async(self, actions):
        if(type(actions) == int):
            for remote in self.remotes:
                remote.send(('step', actions))
        else:
            for remote, action in zip(self.remotes, actions):
                remote.send(('step', action))
        self.waiting = True

    def step_wait(self):
        results = [remote.recv() for remote in self.remotes]
        self.waiting = False
        obs, rews, dones, infos = zip(*results)
        return np.stack(obs), np.stack(rews), np.stack(dones), infos

    def reset(self):
        for remote in self.remotes:
            remote.send(('reset', None))
        return np.stack([remote.recv() for remote in self.remotes])

    def reset_task(self):
        for remote in self.remotes:
            remote.send(('reset_task', None))
        return np.stack([remote.recv() for remote in self.remotes])

    def close(self):
        if self.closed:
            return
        if self.waiting:
            for remote in self.remotes:
                remote.recv()
        for remote in self.remotes:
            remote.send(('close', None))
        for p in self.ps:
            p.join()
            self.closed = True

    def __len__(self):
        return self.nenvs



def shape_list(x):
  """Return list of dims, statically where possible."""
  x = tf.convert_to_tensor(x)

  # If unknown rank, return dynamic shape
  if x.get_shape().dims is None:
    return tf.shape(x)

  static = x.get_shape().as_list()
  shape = tf.shape(x)

  ret = []
  for i, dim in enumerate(static):
    if dim is None:
      dim = shape[i]
    ret.append(dim)
  return ret

def to_float(x):
  """Cast x to float; created because tf.to_float is deprecated."""
  return tf.cast(x, tf.float32)

def cast_like(x, y):
  """Cast x to y's dtype, if necessary."""
  x = tf.convert_to_tensor(x)
  y = tf.convert_to_tensor(y)

  if x.dtype.base_dtype == y.dtype.base_dtype:
    return x

  cast_x = tf.cast(x, y.dtype)
  if cast_x.device != x.device:
    x_name = "(eager Tensor)"
    try:
      x_name = x.name
    except AttributeError:
      pass
    tf.logging.warning("Cast for %s may induce copy from '%s' to '%s'", x_name,
                       x.device, cast_x.device)
  return cast_x

def layer_norm_vars(filters):
  """Create Variables for layer norm."""
  scale = tf.get_variable(
      "layer_norm_scale", [filters], initializer=tf.ones_initializer())
  bias = tf.get_variable(
      "layer_norm_bias", [filters], initializer=tf.zeros_initializer())
  return scale, bias


def layer_norm_compute(x, epsilon, scale, bias, layer_collection=None):
  """Layer norm raw computation."""

  # Save these before they get converted to tensors by the casting below
  params = (scale, bias)

  epsilon, scale, bias = [cast_like(t, x) for t in [epsilon, scale, bias]]
  mean = tf.reduce_mean(x, axis=[-1], keepdims=True)
  variance = tf.reduce_mean(
      tf.squared_difference(x, mean), axis=[-1], keepdims=True)
  norm_x = (x - mean) * tf.rsqrt(variance + epsilon)

  output = norm_x * scale + bias


  return output

def layer_norm(x,
               filters=None,
               epsilon=1e-6,
               name=None,
               reuse=None,
               layer_collection=None):
  """Layer normalize the tensor x, averaging over the last dimension."""
  if filters is None:
    filters = shape_list(x)[-1]
  with tf.variable_scope(
      name, default_name="layer_norm", values=[x], reuse=reuse):
    scale, bias = layer_norm_vars(filters)
    return layer_norm_compute(x, epsilon, scale, bias,
                              layer_collection=layer_collection)

def standardize_images(x):
  """Image standardization on batches and videos."""
  with tf.name_scope("standardize_images", values=[x]):
    x_shape = shape_list(x)
    x = to_float(tf.reshape(x, [-1] + x_shape[-3:]))
    x_mean = tf.reduce_mean(x, axis=[1, 2], keepdims=True)
    x_variance = tf.reduce_mean(
        tf.squared_difference(x, x_mean), axis=[1, 2], keepdims=True)
    num_pixels = to_float(x_shape[-2] * x_shape[-3])
    x = (x - x_mean) / tf.maximum(tf.sqrt(x_variance), tf.rsqrt(num_pixels))
    return tf.reshape(x, x_shape)

def pad_to_same_length(x, y, final_length_divisible_by=1, axis=1):
  """Pad tensors x and y on axis 1 so that they have the same length."""
  if axis not in [1, 2]:
    raise ValueError("Only axis=1 and axis=2 supported for now.")
  with tf.name_scope("pad_to_same_length", values=[x, y]):
    x_length = shape_list(x)[axis]
    y_length = shape_list(y)[axis]
    if (isinstance(x_length, int) and isinstance(y_length, int) and
        x_length == y_length and final_length_divisible_by == 1):
      return x, y
    max_length = tf.maximum(x_length, y_length)
    if final_length_divisible_by > 1:
      # Find the nearest larger-or-equal integer divisible by given number.
      max_length += final_length_divisible_by - 1
      max_length //= final_length_divisible_by
      max_length *= final_length_divisible_by
    length_diff1 = max_length - x_length
    length_diff2 = max_length - y_length

    def padding_list(length_diff, arg):
      if axis == 1:
        return [[[0, 0], [0, length_diff]],
                tf.zeros([tf.rank(arg) - 2, 2], dtype=tf.int32)]
      return [[[0, 0], [0, 0], [0, length_diff]],
              tf.zeros([tf.rank(arg) - 3, 2], dtype=tf.int32)]

    paddings1 = tf.concat(padding_list(length_diff1, x), axis=0)
    paddings2 = tf.concat(padding_list(length_diff2, y), axis=0)
    res_x = tf.pad(x, paddings1)
    res_y = tf.pad(y, paddings2)
    # Static shapes are the same except for axis=1.
    x_shape = x.shape.as_list()
    x_shape[axis] = None
    res_x.set_shape(x_shape)
    y_shape = y.shape.as_list()
    y_shape[axis] = None
    res_y.set_shape(y_shape)
    return res_x, res_y

def make_even_size(x):
  """Pad x to be even-sized on axis 1 and 2, but only if necessary."""
  x_shape = x.get_shape().as_list()
  assert len(x_shape) > 2, "Only 3+-dimensional tensors supported."
  shape = [dim if dim is not None else -1 for dim in x_shape]
  new_shape = x_shape  # To make sure constant shapes remain constant.
  if x_shape[1] is not None:
    new_shape[1] = 2 * int(math.ceil(x_shape[1] * 0.5))
  if x_shape[2] is not None:
    new_shape[2] = 2 * int(math.ceil(x_shape[2] * 0.5))
  if shape[1] % 2 == 0 and shape[2] % 2 == 0:
    return x
  if shape[1] % 2 == 0:
    x, _ = pad_to_same_length(x, x, final_length_divisible_by=2, axis=2)
    x.set_shape(new_shape)
    return x
  if shape[2] % 2 == 0:
    x, _ = pad_to_same_length(x, x, final_length_divisible_by=2, axis=1)
    x.set_shape(new_shape)
    return x
  x, _ = pad_to_same_length(x, x, final_length_divisible_by=2, axis=1)
  x, _ = pad_to_same_length(x, x, final_length_divisible_by=2, axis=2)
  x.set_shape(new_shape)
  return x


def add_timing_signal_nd(x, min_timescale=1.0, max_timescale=1.0e4):
  """Adds a bunch of sinusoids of different frequencies to a Tensor.

  Each channel of the input Tensor is incremented by a sinusoid of a different
  frequency and phase in one of the positional dimensions.

  This allows attention to learn to use absolute and relative positions.
  Timing signals should be added to some precursors of both the query and the
  memory inputs to attention.

  The use of relative position is possible because sin(a+b) and cos(a+b) can be
  expressed in terms of b, sin(a) and cos(a).

  x is a Tensor with n "positional" dimensions, e.g. one dimension for a
  sequence or two dimensions for an image

  We use a geometric sequence of timescales starting with
  min_timescale and ending with max_timescale.  The number of different
  timescales is equal to channels // (n * 2). For each timescale, we
  generate the two sinusoidal signals sin(timestep/timescale) and
  cos(timestep/timescale).  All of these sinusoids are concatenated in
  the channels dimension.

  Args:
    x: a Tensor with shape [batch, d1 ... dn, channels]
    min_timescale: a float
    max_timescale: a float

  Returns:
    a Tensor the same shape as x.
  """
  num_dims = len(x.get_shape().as_list()) - 2
  channels = shape_list(x)[-1]
  num_timescales = channels // (num_dims * 2)
  log_timescale_increment = (
      math.log(float(max_timescale) / float(min_timescale)) /
      (tf.to_float(num_timescales) - 1))
  inv_timescales = min_timescale * tf.exp(
      tf.to_float(tf.range(num_timescales)) * -log_timescale_increment)
  for dim in range(num_dims):
    length = shape_list(x)[dim + 1]
    position = tf.to_float(tf.range(length))
    scaled_time = tf.expand_dims(position, 1) * tf.expand_dims(
        inv_timescales, 0)
    signal = tf.concat([tf.sin(scaled_time), tf.cos(scaled_time)], axis=1)
    prepad = dim * 2 * num_timescales
    postpad = channels - (dim + 1) * 2 * num_timescales
    signal = tf.pad(signal, [[0, 0], [prepad, postpad]])
    for _ in range(1 + dim):
      signal = tf.expand_dims(signal, 0)
    for _ in range(num_dims - 1 - dim):
      signal = tf.expand_dims(signal, -2)
    x += signal
  return x
